In [1]:
import pandas as pd

# Nombres de los archivos parquet
archivos_parquet = ['clean_df_ur.parquet', 'definitivo_df_ui.parquet', 'definitivo_df_sg.parquet']

# Cargar el primer archivo parquet
df_ur = pd.read_parquet('DataSet/' + archivos_parquet[0])

# Cargar el segundo archivo parquet
df_ui = pd.read_parquet('DataSet/' + archivos_parquet[1])

# Cargar el tercer archivo parquet
df_sg = pd.read_parquet('DataSet/' + archivos_parquet[2])

In [2]:
def PlayTimeGenre(genero: str):
    # Fusionar df_ui y df_sg basado en la columna 'item_id' de df_ui y la columna 'id' de df_sg
    merged_df = pd.merge(df_ui, df_sg, left_on='item_id', right_on='id')

    # Filtrar el DataFrame fusionado para incluir solo las filas que contienen el género especificado
    filtered_df = merged_df[merged_df['genres'].str.contains(genero)]
    # Agrupar por año de lanzamiento y sumar el tiempo de juego total para ese año
    hours_by_year = filtered_df.groupby('release_year')['playtime_forever'].sum()

    # Verificar si se encontraron resultados
    if hours_by_year.empty:
        return "El género ingresado no arroja resultados o no corresponde. Vuelva a ingresar."
    else:
        # Encontrar el año con el mayor tiempo de juego total
        max_year = hours_by_year.idxmax()
        return {"Año de lanzamiento con más horas jugadas para Género {}".format(genero): max_year}



In [7]:
# Ejemplo de prueba para PlayTimeGenre
print(PlayTimeGenre("action"))


{'Año de lanzamiento con más horas jugadas para Género action': 2012}


In [4]:
def UserForGenre(genero: str):
    # Fusionar df_ui y df_sg basado en la columna 'item_id' de df_ui y la columna 'id' de df_sg
    merged_df = pd.merge(df_ui, df_sg, left_on='item_id', right_on='id')

    # Filtrar el DataFrame fusionado para incluir solo las filas que contienen el género especificado
    filtered_df = merged_df[merged_df['genres'].str.contains(genero)]

    # Verificar si hay resultados después del filtrado
    if filtered_df.empty:
        return "El género ingresado no arroja resultados o no corresponde. Vuelva a ingresar."

    # Encontrar el usuario con el mayor tiempo total de juego
    top_user = filtered_df.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Agrupar por año de lanzamiento y sumar el tiempo de juego total para ese año para el usuario con más horas jugadas
    hours_by_year = filtered_df[filtered_df['user_id'] == top_user].groupby('release_year')['playtime_forever'].sum().reset_index()

    return {"Usuario con más horas jugadas para Género {}".format(genero): top_user, "Horas jugadas": hours_by_year.to_dict('records')}



In [6]:
# Ejemplo de prueba para UserForGenre
print(UserForGenre("action"))


{'Usuario con más horas jugadas para Género action': 'Sp3ctre', 'Horas jugadas': [{'release_year': 1995, 'playtime_forever': 217.0}, {'release_year': 1999, 'playtime_forever': 44.0}, {'release_year': 2000, 'playtime_forever': 70644.0}, {'release_year': 2001, 'playtime_forever': 13.0}, {'release_year': 2002, 'playtime_forever': 238.0}, {'release_year': 2003, 'playtime_forever': 7673.0}, {'release_year': 2004, 'playtime_forever': 127411.0}, {'release_year': 2005, 'playtime_forever': 21339.0}, {'release_year': 2006, 'playtime_forever': 896.0}, {'release_year': 2007, 'playtime_forever': 112784.0}, {'release_year': 2008, 'playtime_forever': 224.0}, {'release_year': 2009, 'playtime_forever': 108326.0}, {'release_year': 2010, 'playtime_forever': 78083.0}, {'release_year': 2011, 'playtime_forever': 154896.0}, {'release_year': 2012, 'playtime_forever': 378296.0}, {'release_year': 2013, 'playtime_forever': 120306.0}, {'release_year': 2014, 'playtime_forever': 137109.0}, {'release_year': 2015, 'p

In [8]:
def UsersRecommend(año: int):
    # Filtrar el DataFrame df_ur para un año específico, recomendaciones positivas y análisis de sentimientos positivo/neutro
    filtered_df = df_ur[(df_ur['posted_year'] == año) & (df_ur['recommend'] == True) & (df_ur['sentiment_analysis'] > 0)]

    # Verificar si hay resultados después del filtrado
    if filtered_df.empty:
        return "No se encontraron juegos recomendados para el año ingresado."

    # Encontrar los tres juegos con más recomendaciones
    top_games = filtered_df['item_id'].value_counts().head(3).index.tolist()

    # Devolver una lista de los juegos más recomendados, ordenados del más al menos recomendado
    return [{"Puesto {}".format(i+1): game} for i, game in enumerate(top_games)]



In [9]:
# Ejemplo de prueba para UsersRecommend
print(UsersRecommend(2016))


[{'Puesto 1': 730}, {'Puesto 2': 440}, {'Puesto 3': 4000}]


In [10]:
def UsersWorstDeveloper(año: int):
    # Comentar: Unir los DataFrame df_ur y df_sg usando 'item_id' como clave
    merged_df = pd.merge(df_ur, df_sg, left_on='item_id', right_on='id')

    # Comentar: Filtrar el DataFrame combinado para obtener solo entradas del año específico con recomendaciones negativas y sentimiento negativo
    filtered_df = merged_df[(merged_df['posted_year'] == año) & (merged_df['recommend'] == False) & (merged_df['sentiment_analysis'] == 0)]

    # Comentar: Verificar si hay resultados después del filtrado
    if filtered_df.empty:
        return "No se encontraron desarrolladores con recomendaciones negativas para el año ingresado."

    # Comentar: Encontrar los tres desarrolladores con más recomendaciones negativas
    worst_devs = filtered_df['developer'].value_counts().head(3).index.tolist()

    # Comentar: Devolver una lista de los desarrolladores peor valorados, ordenados del peor al menos malo
    return [{"Puesto {}".format(i+1): dev} for i, dev in enumerate(worst_devs)]



In [11]:
# Ejemplo de prueba para UsersWorstDeveloper
print(UsersWorstDeveloper(2014))


[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Bohemia Interactive'}, {'Puesto 3': 'Facepunch Studios'}]


In [12]:
def sentiment_analysis(empresa_desarrolladora: str):
    # Unir df_ur con df_sg utilizando 'item_id' como clave
    merged_df = pd.merge(df_ur, df_sg, left_on='item_id', right_on='id')

    # Filtrar el DataFrame combinado para obtener solo las entradas que corresponden a la empresa desarrolladora especificada
    filtered_df = merged_df[merged_df['developer'] == empresa_desarrolladora]

    # Verificar si hay resultados después del filtrado
    if filtered_df.empty:
        return "No se encontraron datos para la empresa desarrolladora: {}".format(empresa_desarrolladora)

    # Contar la frecuencia de cada valor en la columna 'sentiment_analysis'
    sentiment_counts = filtered_df['sentiment_analysis'].value_counts().to_dict()

    # Devolver un diccionario con el conteo de sentimientos negativos, neutrales y positivos
    return {empresa_desarrolladora: {"Negative": sentiment_counts.get(0, 0), "Neutral": sentiment_counts.get(1, 0), "Positive": sentiment_counts.get(2, 0)}}



In [13]:
# Ejemplo de prueba para sentiment_analysis
print(sentiment_analysis("Valve"))


{'Valve': {'Negative': 1609, 'Neutral': 2570, 'Positive': 5384}}
